In [1]:
import os
import yaml
import pandas as pd
from typing import Any
from typing import Dict
import urllib.parse
import difflib
import re
import requests
import time
import json
import xml.etree.ElementTree as et
from zlib import crc32 
import yaml
import collections
import nltk
from nltk.corpus import stopwords
import numpy as np
import xml.dom.minidom as md
import html
from lxml import etree

In [3]:
def get_website_dois():
    all_website_dois = []
    for file in os.listdir('../data/xml/'):        
        input = '../data/xml/'+ file        
        tree = et.parse(input)
        root = tree.getroot()

        for paper in root.iter('paper'):
            all_website_dois.append(paper.find("doi").text)

    return all_website_dois

a = get_website_dois()

In [5]:
pd.DataFrame(a).to_csv('all_doi.csv', index=False)

In [145]:
import os
import pandas as pd
import xml.etree.ElementTree as et
import html

# print(os.listdir('data/xml/'))

def get_website_dois():
    all_website_dois = []
    for file in os.listdir('../data/xml/'):
        # sheet = pd.DataFrame(columns=["title", "author", "abstract", "url", "pages", "doi", "bibkey"])    
        input = '../data/xml/'+ file
        # output = 'gwf_data_extract/'+file+'all_website_dois.csv'
        tree = et.parse(input)
        root = tree.getroot()

        for paper in root.iter('paper'):
            all_website_dois.append(paper.find("doi").text)

    return all_website_dois

def get_website_bibkeys():
    all_website_bibkeys = []
    files_path = '../data/xml/'
    for file in os.listdir(files_path):
        input = files_path+ file        
        tree = et.parse(input)
        root = tree.getroot()

        for paper in root.iter('paper'):
            all_website_bibkeys.append(paper.find("bibkey").text)

    return all_website_bibkeys

def find_doi_diffs(all_to_compare_file):
    dois = pd.read_csv(all_to_compare_file, header=0).applymap(str.lower).fillna("").values.tolist()
    dois = [item[0] for item in dois]
    websit_dois = get_website_dois()
    print(len(dois))
    print(len(websit_dois))

    added_dois = [x for x in dois if x not in websit_dois]
    # added_dois = [x for x in websit_dois if x not in dois]
    return added_dois


# a = find_doi_diffs('doi_from_USask.csv')




In [39]:
data_url =('https://api.crossref.org/works/{DOI}')
dois = pd.read_csv('extra_publication_cleaned.csv', header=None).iloc[:,0].values.tolist()

with open('result.json', 'r') as openfile:
    detail_dict = json.load(openfile)

for doi in dois:
    if doi not in detail_dict:
        try:
            print(doi)
            url = data_url.replace("{DOI}", doi)
            r = requests.get(url)
            if r.status_code != 200:
                print(r.text)

            data = r.json()
            detail_dict[data['message']['DOI']] = data['message']
        except: 
            print('exception')
            pass

with open('result_added.json', "w") as outfile:
    outfile.write(json.dumps(detail_dict, indent=4)) 


with open('abstract.json', 'r') as openfile:
    abs_dict = json.load(openfile)

data_url='https://api.openalex.org/works/https://doi.org/{DOI}'
for doi in dois:
    if doi not in abs_dict:    
        r = requests.get(url = data_url.replace("{DOI}", doi))
        print(doi)
        if r.status_code != 200:
            print(r.status_code)
        else:
            response = r.json()
            if response['abstract_inverted_index']:   
                abstract = {}         
                for key, indexes in response['abstract_inverted_index'].items():
                    for index in indexes:
                        abstract[index] = key
                
                abs_dict[response['doi'].replace('https://doi.org/', '')] = ' '.join(list(collections.OrderedDict(sorted(abstract.items())).values()))

with open('abstract_added.json', "w") as outfile:
    outfile.write(json.dumps(abs_dict, indent=4))        

10.1001/hyp.14465
404
10.1002/esp.4833
10.1002/esp.4919
10.1002/esp.5140
10.1002/etc.4852
10.1002/etc.5200
10.1002/hyp.10905
10.1002/hyp.10906
10.1002/hyp.10910
10.1002/hyp.11628
10.1002/hyp.13622
10.1002/hyp.13915
10.1002/hyp.13960
10.1002/hyp.14038
10.1002/hyp.14112
10.1002/hyp.14127
10.1002/hyp.14410
10.1002/hyp.14501
10.1002/jeq2.20155
10.1002/joc.6571
10.1002/ppp.2051
10.1002/rcm.8995
10.1002/smll.202101171
10.1002/vzj2.20043
10.1002/vzj2.20101
10.1002/wcc.688
10.1007/s10460-020-10059-z
10.1007/s00382-020-05388-y
10.1007/s00484-020-01867-3
10.1007/s00704-021-03791-x
10.1007/s10584-021-02956-x
10.1007/s10661-020-08570-1
10.1007/s10664-020-09863-2
10.1007/s10933-020-00162-w
10.1007/s11069-020-04425-7
10.1007/s11273-022-09857-5
10.1007/s11625-020-00901-y
10.1007/s12665-016-6027-1
10.1007/s13679-020-00399-6
10.1016/b978-0-12-409548-9.12441-8
10.1016/j.aca.2020.12.056
10.1016/j.agrformet.2019.107774
10.1016/j.agrformet.2020.107910
10.1016/j.agrformet.2021.108407
10.1016/j.apgeochem.202

In [222]:
tag_vol

<Element 'volume' at 0x000001F32435E680>

In [249]:
for year, year_dict in paper_dict.items():
    print(year,len(year_dict))

2018 2
2019 5
2020 1
2017 1
2022 5
2021 4


In [238]:
tree = et.parse("G19.xml")
root = tree.getroot()   
len(root.findall("./volume"))

196

In [239]:
root.findall("./volume")[-1].get('id')

'196'

In [300]:
tag_vol.getchildren

962

In [301]:
xml_folder = os.path.abspath("")

def compute_hash(value: bytes) -> str:
    checksum = crc32(value) & 0xFFFFFFFF
    return f"{checksum:08x}"

def generate_bibkey(title, author, year, bibkey_list):
    author_list = [(item['family'] if 'family' in item else '') for item in author] + [(item['given'] if 'given' in item else '') for item in author]    
    title_list = title.split()
    count=1
    while True:
        bibkey = '-'.join(author_list[:count] + [str(year)] + title_list[:count])
        if bibkey not in bibkey_list: return bibkey
        count += 1

script_dir = os.path.abspath("")
website_dois = get_website_dois()
dois = pd.read_csv(script_dir+'/non_doi_titles.csv', header=None).iloc[:,0].values.tolist()
dois = [item.lower() for item in dois if item not in website_dois] # drop DOIs that already exist

with open(script_dir+'/result.json', 'r') as openfile:     
    json_object = json.load(openfile)

with open(script_dir+'/abstract.json', 'r') as openfile:     
    abstract_dict = json.load(openfile)

paper_dict = {}
json_object = {key:item for key, item in json_object.items() if key in dois}
json_object = {key: item for key, item in json_object.items() if ('author' in item) and (item['title'])} # drop papers without title or author 
    
for key, item in json_object.items():
    try:
        year = item['published']['date-parts'][0][0]
        journal = item['container-title'][0] if item['container-title'] else ' '
        volume = item['volume'] if 'volume' in item else ''
        issue = item['issue'] if 'issue' in item else ''
        pub_key = 'journal: '+journal+' volume: '+volume+' issue: '+issue
        if year not in paper_dict: paper_dict[year]={}
        if pub_key not in paper_dict[year]: paper_dict[year][pub_key]={}
        paper_dict[year][pub_key][key] = item     
    except:            
        pass

bibkey_list = get_website_bibkeys()

for year, year_dict in paper_dict.items():
    tree = et.parse("G"+str(year)[-2:]+".xml")
    root = tree.getroot()

    for index, (volume, item) in enumerate(year_dict.items()):
        first_item = list(item.items())[0][1]
        booktitle = (first_item['container-title'][0] if first_item['container-title'] else ' ')+((', Volume '+first_item['volume']) if 'volume' in first_item else '')+((', Issue '+first_item['issue']) if 'issue' in first_item else '')

        tag_vol = root.find('.//booktitle[.="'+booktitle+'"]/....')
        if tag_vol: # check if volume has existed
            paper_id_ofs = int(tag_vol.findall("./paper")[-1].get('id'))
        else:
            tag_vol = et.Element("volume")
            root.append(tag_vol)
            tag_vol.set('id', str(int(root.findall("./volume")[-2].get('id'))+1))

            tag_meta = et.Element("meta")
            tag_vol.append(tag_meta)
            tag_subelement = et.SubElement(tag_meta, "booktitle")
            tag_subelement.text = booktitle
            tag_subelement = et.SubElement(tag_meta, "publisher")
            tag_subelement.text = first_item['publisher']
            tag_subelement = et.SubElement(tag_meta, "address")
            tag_subelement.text = ""
            tag_subelement = et.SubElement(tag_meta, "year")
            tag_subelement.text = str(year)
            paper_id_ofs = 0

        tmep_dict = year_dict[volume]
        for idx, (key, tmep_dict_item) in enumerate(tmep_dict.items()):
            skip_flag = 0
            for author in tmep_dict_item['author']: # skip papers that don't have author name or family
                if ('given' not in author) or ('family' not in author): skip_flag = 1
            if skip_flag: continue
        
            tag_paper = et.Element("paper")
            tag_vol.append(tag_paper)
            tag_paper.set('id', str(idx+1+paper_id_ofs))
            tag_subelement = et.SubElement(tag_paper, "title")
            tag_subelement.text = tmep_dict_item['title'][0]
            for author in tmep_dict_item['author']:
                tag_subelement = et.SubElement(tag_paper, "author")
                tag_subsubelement = et.SubElement(tag_subelement, "first")
                tag_subsubelement.text = (author['given'] if 'given' in author else '')
                tag_subsubelement = et.SubElement(tag_subelement, "last")
                tag_subsubelement.text = (author['family'] if 'family' in author else '')

            tag_subelement = et.SubElement(tag_paper, "abstract")
            tag_subelement.text = abstract_dict[tmep_dict_item['DOI']] if tmep_dict_item['DOI'] in abstract_dict else ''
            tag_subelement = et.SubElement(tag_paper, "url")
            tag_subelement.text =  f'G{str(year)[-2:]}-{index+1}{(idx+1):03}'
            tag_subelement.set('hash', compute_hash(str.encode(tag_subelement.text)))
        
            if 'page' in tmep_dict_item:
                tag_subelement = et.SubElement(tag_paper, "pages")
                tag_subelement.text = tmep_dict_item['page']

            tag_subelement = et.SubElement(tag_paper, "doi")
            tag_subelement.text = tmep_dict_item['DOI']
            tag_subelement = et.SubElement(tag_paper, "bibkey")
            bibkey = generate_bibkey(tmep_dict_item['title'][0], tmep_dict_item['author'], year, bibkey_list)
            tag_subelement.text = bibkey
            bibkey_list.append(bibkey)

    tree = et.ElementTree(root)    
    file_name = "G"+str(year)[-2:]+"a.xml"   
    with open(file_name, "w", encoding="UTF-8") as f:
        f.write(etree.tostring(etree.XML(et.tostring(root, encoding="UTF-8", xml_declaration=True), parser=etree.XMLParser(remove_blank_text=True))).decode())

    xml_pretty_str = md.parse(file_name)
    xml_pretty_str = xml_pretty_str.toprettyxml(encoding='UTF-8').decode()
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(xml_pretty_str)    

In [302]:
len(json_object)

18

In [312]:
with open('name_variants.yaml', 'r') as file:
    dict_file = yaml.safe_load(file)
for key in json_object:
    for item in json_object[key]['author']:
        if ('given' in item) and ('family' in item): element = {'canonical' : {'first': (item['given'] if 'given' in item else ''), 'last': (item['family'] if 'family' in item else '')}, 'id':(item['given'] if 'given' in item else '').replace('.', '')+'-'+(item['family'] if 'family' in item else '').replace('.', '')}
        if element not in dict_file: dict_file.append(element)

with open('name_variants.yaml', 'w') as file:
    documents = yaml.dump(dict_file, file, default_flow_style=None)

In [310]:
prime_service[5] in dict_file

True

In [304]:
dict_file

[{'canonical': {'first': 'Grant', 'last': 'Ferguson'}, 'id': 'Grant-Ferguson'},
 {'canonical': {'first': 'Jennifer C.', 'last': 'McIntosh'},
  'id': 'Jennifer C-McIntosh'},
 {'canonical': {'first': 'Stephen E.', 'last': 'Grasby'},
  'id': 'Stephen E-Grasby'},
 {'canonical': {'first': 'M. Jim', 'last': 'Hendry'}, 'id': 'M Jim-Hendry'},
 {'canonical': {'first': 'Scott', 'last': 'Jasechko'}, 'id': 'Scott-Jasechko'},
 {'canonical': {'first': 'Matthew B. J.', 'last': 'Lindsay'},
  'id': 'Matthew B J-Lindsay'},
 {'canonical': {'first': 'Elco', 'last': 'Luijendijk'},
  'id': 'Elco-Luijendijk'},
 {'canonical': {'first': 'Serghei A.', 'last': 'Bocaniov'},
  'id': 'Serghei A-Bocaniov'},
 {'canonical': {'first': 'Philippe', 'last': 'Van Cappellen'},
  'id': 'Philippe-Van Cappellen'},
 {'canonical': {'first': 'Donald', 'last': 'Scavia'}, 'id': 'Donald-Scavia'},
 {'canonical': {'first': 'Laura E.', 'last': 'Condon'},
  'id': 'Laura E-Condon'},
 {'canonical': {'first': 'Katherine H.', 'last': 'Marko

In [ ]:
Resource not found. 10.1001/hyp.14465
Resource not found. 10.1016/j.hydrol.2021.126793
Resource not found. 10.20383/101.0275
Resource not found. 10.22004/ag.econ.302498
Resource not found. 10.4230/lipics.swat.2018.12
Resource not found. 10.48550/arxiv.2204.05345
Resource not found. 10.5683/sp2/zrgxq5

In [48]:
def compute_hash(value: bytes) -> str:
    checksum = crc32(value) & 0xFFFFFFFF
    return f"{checksum:08x}"

def generate_bibkey(title, author, year, bibkey_list):
    author_list = [(item['family'] if 'family' in item else '') for item in author] + [(item['given'] if 'given' in item else '') for item in author]    
    title_list = title.split()
    count=1
    while True:
        bibkey = '-'.join(author_list[:count] + [str(year)] + title_list[:count])
        if bibkey not in bibkey_list: return bibkey
        count += 1

with open('result.json', 'r') as openfile:     
    json_object = json.load(openfile)

# with open(abstract_file, 'r') as openfile:     
#     abstract_dict = json.load(openfile)

with_abs, without_abs = 0, 0

for key, item in json_object.items():
    if ('author' not in item) or (not item['title']):
        print(key)

# paper_dict = {}
# json_object = {key: item for key, item in json_object.items() if ('author' in item) and (item['title'])} # drop papers without title or author 
# count = 0
# count1 = 0
# count2 = 0
# no_month = 0 
# for key, item in json_object.items():
#     try:        
#         year = item['published']['date-parts'][0][0]
#         if len(item['published']['date-parts'][0])<2: 
#             no_month += 1
#             print(key, year)

#         count1 += 1
#         journal = item['container-title'][0] if item['container-title'] else ' '
#         count2 += 1
#         volume = item['volume'] if 'volume' in item else ''
#         issue = item['issue'] if 'issue' in item else ''
#         pub_key = 'journal: '+journal+' volume: '+volume+' issue: '+issue
#         if year not in paper_dict: paper_dict[year]={}
#         if pub_key not in paper_dict[year]: paper_dict[year][pub_key]={}
#         paper_dict[year][pub_key][key] = item     
#     except:        
#         pass

10.5194/tc-14-2795-2020-supplement
10.5194/hess-24-2731-2020-supplement
10.5194/hess-24-2141-2020-supplement
10.1175/bams-d-18-0268.1
10.5194/bg-17-361-2020-supplement
10.1029/tr002i001p00002


In [45]:
len(json_object)

977

In [7]:
import xml.dom.minidom as md

xml_fname = '''<?xml version='1.0' encoding='UTF-8'?>
<collection id="G14"><volume id="1"><meta><booktitle>Environmental Health Insights, Volume 8</booktitle><publisher>SAGE Publications</publisher><address /><year>2014</year></meta><paper id="1"><title>A Framework for Building Research Partnerships with First Nations Communities</title><author><first>Lalita</first><last>Bharadwaj</last></author><abstract>Solutions to complex health and environmental issues experienced by First Nations communities in Canada require the adoption of collaborative modes of research. The traditional “helicopter” approach to research applied in communities has led to disenchantment on the part of First Nations people and has impeded their willingness to participate in research. University researchers have tended to develop projects without community input and to adopt short term approaches to the entire process, perhaps a reflection of granting and publication cycles and other realities of academia. Researchers often enter communities, collect data without respect for local culture, and then exit, having had little or no community interaction or consideration of how results generated could benefit communities or lead to sustainable solutions. Community-based participatory research (CBPR) has emerged as an alternative to the helicopter approach and is promoted here as a method to research that will meet the objectives of both First Nations and research communities. CBPR is a collaborative approach that equitably involves all partners in the research process. Although the benefits of CBPR have been recognized by segments of the University research community, there exists a need for comprehensive changes in approaches to First Nations centered research, and additional guidance to researchers on how to establish respectful and productive partnerships with First Nations communities beyond a single funded research project. This article provides a brief overview of ethical guidelines developed for researchers planning studies involving Aboriginal people as well as the historical context and principles of CBPR. A framework for building research partnerships with First Nations communities that incorporates and builds upon the guidelines and principles of CBPR is then presented. The framework was based on 10 years’ experience working with First Nations communities in Saskatchewan. The framework for research partnership is composed of five phases. They are categorized as the pre-research, community consultation, community entry, research and research dissemination phases. These phases are cyclical, non-linear and interconnected. Elements of, and opportunities for, exploration, discussion, engagement, consultation, relationship building, partnership development, community involvement, and information sharing are key components of the five phases within the framework. The phases and elements within this proposed framework have been utilized to build and implement sustainable collaborative environmental health research projects with Saskatchewan First Nations communities.</abstract><url hash="0cb058e8">G14-1001</url><pages>EHI.S10869</pages><doi>10.4137/ehi.s10869</doi><bibkey>Bharadwaj-2014-A</bibkey></paper></volume></collection>'''

uglyxml = '<?xml version="1.0" encoding="UTF-8" ?><employees><employee><Name>Leonardo DiCaprio</Name></employee></employees>'

# xml = xml.dom.minidom.parseString(xml_fname)
xml = md.parse('G21.xml')

xml_pretty_str = xml.toprettyxml(encoding="utf-8").decode()
# print(xml_pretty_str)

with open("test.xml", "w", encoding="utf-8") as f:
    f.write(xml_pretty_str)


In [3]:
string_check = re.compile("[@_!#$%^&*()<>?/\|}{~:-]")
nltk.download("stopwords")
list_stopwords = list(stopwords.words("english"))
paper_url = "https://api.semanticscholar.org/v1/paper/"
search_url = "https://api.semanticscholar.org/graph/v1/paper/search?"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Morteza1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
query = "+".join(pub.split()).lower()                    
payload = {'offset': 0, 'limit': 10,'query': query}

PARAMS = urllib.parse.urlencode(payload, safe=':+')

headers={'x-api-key':'LPkwK92ydta0i2EY5UB8fgnVhoLPZb72T3p3TCF1'}

r = requests.get(url=search_url, params=PARAMS, headers=headers)

In [12]:
import pandas as pd
paper_df = pd.read_csv('DOI_all.csv', header=0).fillna("")

In [29]:
# paper_df.drop_duplicates()
for item in paper_df['doi']:
    if '10.3390/SU13042415' in item: print(item)

10.3390/SU13042415


In [33]:
paper_df['doi'] =paper_df['doi'].str.lower().drop_duplicates()
paper_df['doi']

0         10.3390/agriculture11080739
1                  10.1002/agj2.21056
2            10.3390/hydrology8010001
3             10.2489/jswc.2021.00099
4       10.1080/07011784.2022.2032365
                    ...              
1023                              NaN
1024                   10.1101/230722
1025                              NaN
1026        10.5194/essd-14-5139-2022
1027    10.1016/j.jhydrol.2022.128711
Name: doi, Length: 1028, dtype: object

In [19]:
paper_df = pd.read_csv('GWF_all.csv', header=0, encoding='ascii', encoding_errors='ignore').fillna("")

paper_list = [None] * len(paper_df)
# doi_list = [None] * len(paper_df)
# score = [None] * len(paper_df)
title = [None] * len(paper_df)
title_after = [None] * len(paper_df)
similarity_score = [None] * len(paper_df)
# query = [None] * len(paper_df)

result_df = pd.DataFrame()

for i in range(len(paper_df)):
    publication = paper_df.loc[i][0]
    pub = re.split("([Dd][Oo][Ii])", publication)
    if len(pub) < 2:
        paper_list[i] = pub[0]
        
        pub = re.split('[,."]', pub[0])
        pub = pub[np.argmax(np.array([len(item) for item in pub if all(word not in item for word in ['Proceedings', 'Conference', 'Journal'])]))]

        pub = pub.replace("'", "").replace('"', '').replace(':', '').replace('-', ' ')
        title[i] = pub
        pub = pub.split()

        query_len = len(pub)
        
        query = "+".join(pub).lower()                    
        payload = {'offset': 0, 'limit': 10,'query': query,}

        PARAMS = urllib.parse.urlencode(payload, safe=':+')
        headers={'x-api-key':'LPkwK92ydta0i2EY5UB8fgnVhoLPZb72T3p3TCF1'}        
        r = requests.get(url=search_url, params=PARAMS, headers=headers)
        if r.status_code!=200: print("r: " + str(r.content))
        
        try:            
            data = r.json()            
            score = []
            for paper in data["data"]:
                temp = difflib.SequenceMatcher(None, paper["title"], ' '.join(pub))
                a = temp.get_matching_blocks()
                score.append(temp.ratio())

            url2 = paper_url + data["data"][score.index(max(score))]["paperId"]
            similarity_score[i] = max(score)
            title_after[i] = data["data"][score.index(max(score))]["title"]

            r2 = requests.get(url=url2, headers=headers)
            if r2.status_code!=200: print("r2: " + str(r2.content))
            data2 = r2.json()
            paper_doi = data2["doi"]
        except:        
            paper_doi = None
            similarity_score[i] = 'exception'
            title_after[i] = 'exception'
            print("None doi")
        
        # print('paper_list:', paper_list[i])        
        # print('title:', title[i])
        # print('title_after:', title_after[i])
        print(i)

    if i==100: break    

result_df['paper'] = paper_list
result_df['title'] = title
result_df['title_after'] = title_after
result_df['similarity_score'] = similarity_score

result_df.to_csv('test.csv', index=False)

1
None doi
4
None doi
14
21
22
23
24
25
27
28
29
30
31
32
33
34
35
36
37
38
39
40
None doi
41
42
43
44
45
46
48
49
50
51
None doi
63
64
None doi
71
81
83
84
85
86
91
96
97
99


In [5]:
pub = re.split('[,."]', 'Noronha, N., Avarino, A., Balakumar, S., Toy, K., Smith, S., Wekerle, C., Martin-Hill, D., Lookinghorse, M., Drossos, A., Niec, A., Jacobs, B., Thomasen, K., & Lokker, C. â€œMental Health Mobile Applications Developed for Indigenous Communities in Canada: A Scoping Reviewâ€ Canadian Journal of Community Mental Health 41 (1): 1 - 5, 2022. https://')
a = pub
b = np.array([len(item) for item in pub if all(word not in item for word in ['Proceedings', 'Conference', 'Journal'])])
pub = pub[np.argmax(np.array([len(item) for item in pub if all(word not in item for word in ['Proceedings', 'Conference', 'Journal'])]))]
pub

' Lookinghorse'

In [22]:
publication = "McCarter CPR, Wilkinson SL, Moore PA, Waddington JM. 2021. Ecohydrological trade-offs from multiple peatland disturbances: The interactive effects of drainage, harvesting, restoration and wildfire in a Southern Ontario bog. Journal of Hydrologyâ€¯601: 126793, "

pub = re.split("[,.:]", publication)
pub = pub[np.argmax(np.array([len(item) for item in pub]))]

pub = re.sub("([A-Z][.][,])", "", pub)
pub = re.sub("([A-Z][.][A-Z][.])", "", pub).replace(".", "").replace('"', "")
pub = re.sub("([A-Z][.])", "", pub).replace("’", "").replace("'", "")
pub = re.sub("([(][0-9]{4}[)])", "", pub)

pub = [i for item in pub.split(",") for i in item.strip().split()]
pub = [item for item in pub if item.lower() not in list_stopwords and len(item) > 2]

pub = [
    item
    for item in pub
    if (string_check.search(item) == None) and not bool(re.search(r"\d", item))
]

i = 6
while True:
    try:
        query = "+".join(pub[:i]).lower()
        break
    except:
        i -= 1
            
payload = {'offset': 0, 'limit': 50,'query': query}

PARAMS = urllib.parse.urlencode(payload, safe=':+')
r = requests.get(url=search_url, params=PARAMS)
if r.status_code!=200: print("r: " + str(r.content))

try:
    data = r.json()
    if data["total"] > 1:
        score = []
        for paper in data["data"]:
            temp = difflib.SequenceMatcher(None, paper["title"], ' '.join(pub))
            a = temp.get_matching_blocks()
            score.append(temp.ratio())

        url2 = paper_url + data["data"][score.index(max(score))]["paperId"]
        similarity_score = max(score)
        title = data["data"][score.index(max(score))]["title"]
    else:
        url2 = paper_url + data["data"][0]["paperId"]
        similarity_score = difflib.SequenceMatcher(None, data["data"][0]["title"], ' '.join(pub)).ratio() + 5
        title = data["data"][0]["title"]

    r2 = requests.get(url=url2)
    if r2.status_code!=200: print("r2: " + str(r2.content))
    data2 = r2.json()
    paper_doi = data2["doi"]
except:        
    paper_doi = None
    similarity_score = 'exception'
    title = 'exception'
    print("None doi")

return paper_doi, similarity_score, title, query


In [111]:
import os
import sys
import pandas as pd
import re
import requests
import numpy as np
import time
import json
import xml.etree.ElementTree as et
from zlib import crc32 
import html

In [90]:
sheet = pd.DataFrame(columns=["title", "author", "abstract", "url", "pages", "doi", "bibkey"])
tree = et.parse('G17.xml')
root = tree.getroot()

for paper in root.iter('paper'):    
    paper_dict = {}
    paper_dict["title"] = paper.find("title").text
    paper_dict["author"] = ""
    for author in paper.findall("author"):
        paper_dict["author"] = paper_dict["author"] + author.find("first").text + " " + author.find("last").text + ", "
    paper_dict["abstract"] = paper.find("abstract").text
    paper_dict["url"] = paper.find("url").text       
    paper_dict["pages"] = paper.find("pages").text if paper.find("pages") is not None else ""
    paper_dict["doi"] = paper.find("doi").text
    paper_dict["bibkey"] = paper.find("bibkey").text

    sheet = pd.concat([sheet, pd.DataFrame([paper_dict.values()], columns=sheet.columns)], ignore_index=True)

sheet.to_csv("G17.csv", index=False)

In [ ]:
tree = et.parse('G18.xml')
root = tree.getroot()

for paper in root.iter("paper"):
    abstract = paper.find("abstract").text
    paper.find("abstract").text = html.unescape(abstract)
    print(abstract)

with open ("test.xml", "wb") as files :
    tree.write(files, encoding='UTF-8', xml_declaration=True, method='xml')

In [122]:
tree = et.parse('G18.xml')
for element in tree.iter():
    text = element.text
    if text: element.text = html.unescape(text)

with open ("test.xml", "wb") as files :
    tree.write(files, encoding='UTF-8', xml_declaration=True, method='xml')

In [121]:
if not text: print("aas")

aas


In [94]:
import html
print(html.unescape('&#x0025;682m'))

%682m


In [28]:
for item in author:
    print(item)

In [2]:
with open('result.json', 'r') as openfile:     
    json_object = json.load(openfile)

In [3]:
paper_dict = {}
for idx, key in enumerate(json_object):
    if json_object[key]['year'] not in paper_dict: paper_dict[json_object[key]['year']]={}
    paper_dict[json_object[key]['year']][key] = json_object[key]        

In [55]:
journals_list = []

for k, paper in paper_dict.items():
    for key, item in paper.items():#['165c581c0ed8f39eca705f61e6b05dc353fce909']['journal']        
        # try: journals_list.append([item['journal']['name'], item['externalIds']['DOI'], item['year']])
        break
        try: journals_list.append(item['journal']['name'])
        except: pass

journals_list


[]

In [22]:
publication = "Ronnquist, A.L., and Westbrook, C.J. 2021. Beaver dams: how structure, flow state, and landscape setting regulate water storage and release. Science of the Total Environment, 785: 147333, doi: https://doi.org/10.1016/j.scitotenv.2021.147333.Â "
pub = re.split("([Dd][Oo][Ii])", publication)
doi = (pub[-1].split()[0].replace('.org/', '') if pub[-1].startswith('.org/') else pub[2].replace(':', '').split()[0]) if len(pub) >= 3 else None 

In [63]:
paper_list = [None] * len(paper_df)
doi_list = [None] * len(paper_df)
for i in range(len(paper_df)):
    publication = paper_df.loc[i][0]
    pub = re.split("([Dd][Oo][Ii])", publication)
    doi_list[i] = (pub[2].split()[0].replace('.org/', '') if pub[2].startswith('.org/') else pub[2].replace(':', '').split()[0]) if len(pub) == 3 else None    
    
    if doi_list[i]: 
        if requests.get(url='https://api.semanticscholar.org/v1/paper/' + doi_list[i]).status_code != 200:
            doi_list[i] = doi_list[i][:-1] if requests.get(url='https://api.semanticscholar.org/v1/paper/' + doi_list[i][:-1]).status_code == 200 else None
                
    paper_list[i] = pub[0]

paper_df['doi'] = doi_list
paper_df['paper'] = paper_list

10,1016/j.dendro.2017.01.003.
10.5194/hess-2017-531.
10.1016/j.jhy
10.2134/jed2017.10.0398.


In [98]:
r = requests.get(url =('https://api.semanticscholar.org/graph/v1/paper/10.5194/hess-2018-56?fields=paperId,'
    'externalIds,url,title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,isOpenAccess,openAccessPdf,'
    'fieldsOfStudy,s2FieldsOfStudy,publicationTypes,publicationDate,journal,citationStyles,authors,citations,references'))
r.status_code


200

In [99]:
r.json()

{'paperId': 'e7ca1d33678d2efe8cf215a350e8830842d4e2f9',
 'externalIds': {'DOI': '10.5194/hess-2018-56', 'CorpusId': 201632713},
 'url': 'https://www.semanticscholar.org/paper/e7ca1d33678d2efe8cf215a350e8830842d4e2f9',
 'title': 'Combined Impacts of ENSO and MJO on the 2015 Growing 1 Season Drought on the Canadian Prairies 2',
 'abstract': '1Global Institute for Water Security, University of Saskatchewan, Saskatoon, Saskatchewan, Canada S7N3H5 4 2Institute of Space and Atmospheric Studies, University of Saskatchewan, Saskatoon, Saskatchewan, Canada 5 3National Hydrology Research Center, Environment and Climate Change Canada, Saskatoon, SK, Canada 6 Correspondence to: Dr. Yanping Li (yanping.li@usask.ca); Dr. Zhenhua Li (zhenhua.li@usask.ca) 7 Abstract 8',
 'venue': '',
 'year': 2018,
 'referenceCount': 38,
 'citationCount': 1,
 'influentialCitationCount': 0,
 'isOpenAccess': True,
 'openAccessPdf': {'url': 'https://www.hydrol-earth-syst-sci.net/22/5057/2018/hess-22-5057-2018.pdf',
  'st

In [78]:
data = r.json()
type(data)
json_object = json.dumps(data, indent=4)
 
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [81]:
with open('sample.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)

In [262]:
from zlib import crc32 

def compute_hash(value: bytes) -> str:
    checksum = crc32(value) & 0xFFFFFFFF
    return f"{checksum:08x}"

print(compute_hash(str.encode("https://aclanthology.org/J19-1001.pdf")))
# "J19-1001"

bb7abe5b


In [320]:
import yaml

dict_file = [{'canonical' : {'first': 'Maricor', 'last': 'Arlos'}, 'id':'654'}]


# dict_file = dict(
#     A = 'a',
#     B = dict(
#         C = 'c',
#         D = 'd',
#         E = 'f',
#     )
# )

with open(r'store_file.yaml', 'w') as file:
    documents = yaml.dump(dict_file, file, default_flow_style=None)

In [315]:
dict_file = []
for key in json_object:
    for item in json_object[key]['authors']:
        element = {'canonical' : {'first': item['name'].split(' ',1)[0], 'last': item['name'].split(' ',1)[-1]}, 'id':item['name'].replace('.', '').replace(' ', '-')}
        if element not in dict_file: dict_file.append(element)

len(dict_file)

3950

In [341]:
r = requests.get(url =("https://api.crossref.org/works/10.1007/s10664-020-09921-9"))

In [342]:
r.json()['message']['title']

['ID-correspondence: a measure for detecting evolutionary coupling']

In [350]:
import difflib
difflib.SequenceMatcher(None,  'Facilitating Asynchronous Collaboration in Scientific Workflow Composition Using Provenance',"facilitating collaborative scientific workflow composition using Provenance").ratio()

0.8192771084337349

In [347]:
pub = re.split("([,.:])", 'Cynthia ST, Roy B and *Mondal D, "Feature Transformation for Improved Software Bug Detection Models", ACM 15th Innovation in Software Engineering Conference (ISEC 2022), Article 16, pp. 1-10, DA-IICT Gandhinagar, February 2022. ')
pub = pub[np.argmax(np.array([len(item) for item in pub]))]
pub

' "Feature Transformation for Improved Software Bug Detection Models"'

In [355]:
import difflib

string1 = "Carbon dioxide efflux and ecosystem metabolism of small forest lakes"
string2 = "Ecosystem influences carbon water dynamics two temperate forage crops"

temp = difflib.SequenceMatcher(None,string1 ,string2)

print(temp.get_matching_blocks())
print('Similarity Score: ',temp.ratio())

[Match(a=0, b=0, size=2), Match(a=2, b=9, size=1), Match(a=5, b=12, size=9), Match(a=14, b=25, size=6), Match(a=20, b=31, size=0)]
Similarity Score:  0.7058823529411765


In [86]:
data_url =('    {DOI}')

all_data_dict = {}

paper_df = pd.read_csv('DOI_all.csv', header=0).fillna("")['doi']

for i in range(len(paper_df)):
    print(i)
    try:
        url = data_url.replace("{DOI}", paper_df[i])
        r = requests.get(url)
        if r.status_code != 200:
            print(r.text)

        data = r.json()
        all_data_dict[data['message']['DOI']] = data['message']
    except: 
        print('exception')
        pass    
    
    if i==100:
        json_object = json.dumps(all_data_dict, indent=4)
        with open('result.json', "w") as outfile:
            outfile.write(json_object)

json_object = json.dumps(all_data_dict, indent=4)
with open('result.json', "w") as outfile:
    outfile.write(json_object)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
Resource not found.
exception
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269


In [41]:
json_objecta = {key: itema for key, itema in json_object.items() if 'author' in itema}    

In [58]:
len(json_object)

1034

In [87]:
with open('result.json', 'r') as openfile:     
    json_object = json.load(openfile)

In [85]:
paper_dict = {}
json_object = {key: item for key, item in json_object.items() if ('author' in item) and (item['title'])} # drop papers without title or author 
for key, item in json_object.items():
    try:
        year = item['published']['date-parts'][0][0]
        journal = item['container-title'][0]
        volume = item['volume'] if 'volume' in item else ''
        issue = item['issue'] if 'issue' in item else ''
        pub_key = 'journal: '+journal+' volume: '+volume+' issue: '+issue
        if year not in paper_dict: paper_dict[year]={}
        if pub_key not in paper_dict[year]: paper_dict[year][pub_key]={}
        paper_dict[year][pub_key][key] = item     
    except:
        pass

In [77]:
def compute_hash(value: bytes) -> str:
    checksum = crc32(value) & 0xFFFFFFFF
    return f"{checksum:08x}"

def generate_bibkey(title, author, year, bibkey_list):
    author_list = [(item['family'] if 'family' in item else '') for item in author] + [(item['given'] if 'given' in item else '') for item in author]    
    title_list = title.split()
    count=1
    while True:
        bibkey = '-'.join(author_list[:count] + [str(year)] + title_list[:count])
        if bibkey not in bibkey_list: return bibkey
        count += 1
        print('bibkey', bibkey)

for year, year_dict in paper_dict.items():
    tag_collection = et.Element("collection")
    tag_collection.set('id', "G"+str(year)[-2:])

    bibkey_list = []

    for index, (volume, item) in enumerate(year_dict.items()):
        tag_vol = et.Element("volume")
        tag_collection.append(tag_vol)
        tag_vol.set('id', str(index+1))

        first_item = list(item.items())[0][1]

        tag_meta = et.Element("meta")
        tag_vol.append(tag_meta)    
        tag_subelement = et.SubElement(tag_meta, "booktitle")
        tag_subelement.text = first_item['container-title'][0]+((', Volume '+first_item['volume']) if 'volume' in first_item else '')+((', Issue '+first_item['issue']) if 'issue' in first_item else '')
        tag_subelement = et.SubElement(tag_meta, "publisher")
        tag_subelement.text = first_item['publisher']
        tag_subelement = et.SubElement(tag_meta, "address")
        tag_subelement.text = ""
        tag_subelement = et.SubElement(tag_meta, "year")
        tag_subelement.text = str(year)

        # tag_frontmatter = et.Element("frontmatter")
        # tag_vol.append(tag_frontmatter)
        # tag_subelement = et.SubElement(tag_frontmatter, "url")
        # tag_subelement.text = str(year)+".GWF-1.0"   
        # tag_subelement.set('hash', compute_hash(str.encode(tag_subelement.text)))
        # tag_subelement = et.SubElement(tag_frontmatter, "bibkey")
        # tag_subelement.text = "G"+str(year)[-2:] +'-'+ str(year) + '-water'        

        tmep_dict = year_dict[volume]
        for idx, (key, tmep_dict_item) in enumerate(tmep_dict.items()):
            tag_paper = et.Element("paper")
            tag_vol.append(tag_paper)
            tag_paper.set('id', str(idx+1))
            tag_subelement = et.SubElement(tag_paper, "title")
            tag_subelement.text = tmep_dict_item['title'][0]
            for author in tmep_dict_item['author']:
                tag_subelement = et.SubElement(tag_paper, "author")
                tag_subsubelement = et.SubElement(tag_subelement, "first")
                tag_subsubelement.text = (author['given'] if 'given' in author else '')
                tag_subsubelement = et.SubElement(tag_subelement, "last")
                tag_subsubelement.text = (author['family'] if 'family' in author else '')

            tag_subelement = et.SubElement(tag_paper, "abstract")
            tag_subelement.text = tmep_dict_item['abstract'] if 'abstract' in tmep_dict_item else ''
            tag_subelement = et.SubElement(tag_paper, "url")
            tag_subelement.text =  f'G{str(year)[-2:]}-{index+1}{(idx+1):03}'
            tag_subelement.set('hash', compute_hash(str.encode(tag_subelement.text)))
            
            if 'page' in tmep_dict_item:
                tag_subelement = et.SubElement(tag_paper, "pages")
                tag_subelement.text = tmep_dict_item['page']

            tag_subelement = et.SubElement(tag_paper, "doi")
            tag_subelement.text = tmep_dict_item['DOI']
            tag_subelement = et.SubElement(tag_paper, "bibkey")
            bibkey = generate_bibkey(tmep_dict_item['title'][0], tmep_dict_item['author'], year, bibkey_list)            
            tag_subelement.text = bibkey
            bibkey_list.append(bibkey)

    # tree = et.ElementTree(tag_collection)    
    # with open ("G"+str(year)[-2:]+".xml", "wb") as files :
    #     tree.write(files, encoding='UTF-8', xml_declaration=True)

bibkey Liu-2021-Phosphorus
bibkey Clark-2021-The
bibkey Bhattacharjee-2022-Supporting
bibkey Huang-2022-A
bibkey Liu-2019-Impacts
bibkey Lindenschmidt-2019-A
bibkey Robinne-2019-A


In [104]:
dict_file = []
for key in json_object:
    for item in json_object[key]['author']:
        if ('given' in item) and ('family' in item): element = {'canonical' : {'first': (item['given'] if 'given' in item else ''), 'last': (item['family'] if 'family' in item else '')}, 'id':(item['given'] if 'given' in item else '').replace('.', '')+'-'+(item['family'] if 'family' in item else '').replace('.', '')}
        if element not in dict_file: dict_file.append(element)

In [103]:
a = ''.replace('.','')
a

''

In [107]:
dict_file = []
json_object = {key: item for key, item in json_object.items() if ('author' in item) and (item['title'])} # drop papers without title or author 
for key in json_object:
    for item in json_object[key]['author']:
        if ('given' in item) and ('family' in item): dict_file.append((item['given'], item['family']))
        else: print("- not complete  ", json_object[key]['published']['date-parts'][0][0], item, json_object[key]['title'])

# dict_file        

- not complete   2022 {'name': 'Six Nations Youth Mental Wellness Advisory Committee', 'sequence': 'additional', 'affiliation': []} ['Indigenous youth mental wellness and the adaptation of the JoyPopTM app (Preprint)']
- not complete   2018 {'name': 'WCRP Global Sea Level Budget Group', 'sequence': 'first', 'affiliation': []} ['Global sea-level budget 1993–present']
- not complete   2019 {'name': '1 Department of Computer Science, University of Saskatchewan, 110 Science Place, Saskatoon, SK, S7N 5C9, Canada', 'sequence': 'first', 'affiliation': []} ['On theoretical upper limits for valid timesteps of implicit ODE methods']
- not complete   2019 {'name': '2 Department of Mathematics and Statistics, University of Saskatchewan, 106 Wiggins Road, Saskatoon, SK, S7N 5E6, Canada', 'sequence': 'additional', 'affiliation': []} ['On theoretical upper limits for valid timesteps of implicit ODE methods']
- not complete   2019 {'name': 'Global Institute for Water Security, University of Saskatchew

In [23]:
paper_df = list(pd.read_csv('DOI_all.csv', header=0).fillna("")['doi'])

In [78]:
# r = requests.get('https://api.crossref.org/works/10.1016/j.jhydrol.2022.128711')
r = requests.get('https://api.crossref.org/works/10.5194/essd-14-5139-2022')
r = r.json()['message']

In [64]:
data_url='https://api.openalex.org/works/https://doi.org/{DOI}'
response_dict = {}
for idx, item in enumerate(set(paper_df)):    
    r = requests.get(url = data_url.replace("{DOI}", item))
    print(idx)
    if r.status_code != 200:
        print(r.status_code)
    else:
        response = r.json()
        if response['abstract_inverted_index']:   
            abstract = {}         
            for key, indexes in response['abstract_inverted_index'].items():
                for index in indexes:
                    abstract[index] = key
            
            response_dict[response['doi'].replace('https://doi.org/', '')] = ' '.join(list(collections.OrderedDict(sorted(abstract.items())).values()))



0


In [68]:
with open('abstract.json', 'r') as openfile:     
            json_object = json.load(openfile)            

In [3]:
paper_df = pd.read_excel("GWF_all.xlsx", header=0).fillna("")

In [6]:
a = re.split("([Dd][Oo][Ii])", 'asdfdoi')
a

['asdf', 'doi', '']

In [10]:
count = 0
doi_list = [None] * len(paper_df)
for i in range(len(paper_df)):
    publication = paper_df.loc[i][0]
    pub = re.split("([Dd][Oo][Ii])", publication)
    doi_list[i] = (pub[-1].split()[0].replace('.org/', '') if pub[-1].startswith('.org/') else pub[-1].replace(':', '').split()[0]) if len(pub) >= 3 else None
    if len(pub) == 1:
        count += 1


In [12]:
len(set(doi_list))

942

In [9]:
import yaml

def read_yaml_file(filename):
    with open(filename, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

def print_common_parts(file1, file2):
    data1 = read_yaml_file(file1)
    data2 = read_yaml_file(file2)
    
    if isinstance(data1, dict) and isinstance(data2, dict):
        common_items = {k: v for k, v in data1.items() if k in data2 and data2[k] == v}
        print("Common parts:", common_items)
    elif isinstance(data1, list) and isinstance(data2, list):
        common_elements = [element for element in data1 if element in data2]
        print("Common parts:", common_elements)

# usage
print_common_parts('../data/yaml/name_variants.yaml', 'name_variants.yaml')


Common parts: []


In [1]:
import pandas as pd

doi_prj_df = pd.read_excel('doi2projects.xlsx')

In [2]:
a = doi_prj_df['Project Name'].value_counts()
prj_dict = {}
cnt = 1
for item, count in a.iteritems():
    prj_dict[item] = "prj"+str(cnt)
    cnt += 1
doi_prj_df['prj'] = doi_prj_df['Project Name'].map(prj_dict)

In [5]:
doi_prj_df.to_excel('doi2projects1.xlsx', index=False)

In [7]:
for key, value in prj_dict.items():
    print(f'"{value}"', end=' ')

print()
for key, value in prj_dict.items():
    print(f'"{key}"', end=' ')    

"prj1" "prj2" "prj3" "prj4" "prj5" "prj6" "prj7" "prj8" "prj9" "prj10" "prj11" "prj12" "prj13" "prj14" "prj15" "prj16" "prj17" "prj18" "prj19" "prj20" "prj21" "prj22" "prj23" "prj24" "prj25" "prj26" "prj27" "prj28" "prj29" "prj30" "prj31" "prj32" "prj33" "prj34" "prj35" "prj36" "prj37" "prj38" "prj39" "prj40" "prj41" "prj42" "prj43" "prj44" "prj45" 
"Northern Water Futures" "Integrated Modelling Program for Canada (IMPC)" "Core Modelling & Forecasting Team" "Mountain Water Futures" "Core Computer Science Team" "Boreal Water Futures: Canada’s Boreal Wildlands-Society-Water Nexus" "Sensors and Sensing Systems for Water Quality Monitoring" "Agricultural Water Futures" "Southern Forests Water Futures" "Lake Futures: Enhancing Adaptive Capacity and Resilience of Lakes and their Watersheds" "(eDNA) Next Generation Solutions to Ensure Healthy Water Resources for Future Generations" "Climate-Related Precipitation Extremes" "Transformative Sensor Technologies and Smart Watersheds for Canadian W

In [5]:
import pandas as pd

# Read the CSV file
doi_prj_df = pd.read_csv("csv_all.csv")

# Select the relevant columns and create a new DataFrame
output_df = doi_prj_df[['project', 'doi']].copy()
output_df.rename(columns={'project': 'Project Name', 'doi': 'DOI'}, inplace=True)

# Fill NaN values in 'Project Name' column with "Others"
output_df['Project Name'].fillna("Others", inplace=True)

output_df['prj'] = ""  # Add an empty 'prj' column

# Save the output to an Excel file
output_df.to_excel('doi2projects.xlsx', index=False)


/tmp/ipykernel_34364/2367267143.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  output_df['Project Name'].fillna("Others", inplace=True)
